In [1]:
import os
import pandas as pd
import numpy as np
from sklearn import metrics,preprocessing
from sklearn.cross_validation import train_test_split
import re

/Users/ruizhi/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import os
import pandas as pd
import numpy as np
import xgboost

from sklearn import metrics,preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, fbeta_score

from nltk.corpus import stopwords

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
train_data = pd.read_csv("input/train_v2.csv")
test_data = pd.read_csv("input/test_v2.csv")

In [4]:
train_data.head()

,article_id,title,url,publisher,hostname,timestamp,category
0,1,Forex - Pound drops to one-month lows against ...,http://www.nasdaq.com/article/forex-pound-drop...,NASDAQ,www.nasdaq.com,1.390000e+12,4
1,2,Hertz to Exit Equipment Rental Business in $2....,http://www.foxbusiness.com/industries/2014/03/...,Fox Business,www.foxbusiness.com,1.400000e+12,2
2,3,Gold ETF inflows return,http://www.resourceinvestor.com/2014/03/09/gol...,Resource Investor,www.resourceinvestor.com,1.390000e+12,4
3,4,"Hackers call Mt. Gox CEO a liar, say he still ...",http://bgr.com/2014/03/10/mt-gox-fraud-accusat...,BGR,bgr.com,1.390000e+12,4
4,5,Gold Climbs To Near 6-Month High On Concerns A...,http://www.forbes.com/sites/kitconews/2014/03/...,Forbes,www.forbes.com,1.390000e+12,4


In [5]:
# with open('train_full.txt', 'w',errors='ignore') as myfile:
#     for i in range(len(train_data['category'])):
#         filename='raw_train/'+str(i+1)+'.txt'
#         if os.path.exists(filename):
#             with open(filename, 'r',errors='ignore') as data_file:
#                 data=data_file.read().replace('\n', '')
#             myfile.write(train_data['title'][i]+" "+data+ '\n')
#         else:
#             myfile.write(train_data['title'][i]+ '\n')

# #v2: remove numbers and punctuations
# whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ')
# with open('train_full_clean.txt', 'w',errors='ignore') as myfile:
#     for i in range(len(train_data['category'])):
#         filename='raw_train/'+str(i+1)+'.txt'
#         title=train_data['title'][i]
#         title = re.sub(r"[,:.;@#\?\-\%\/!&$]+\ *", " ", title)
#         title = ''.join(filter(whitelist.__contains__, title)).lower()
#         title = ' '.join(title.split())
#         if os.path.exists(filename):
#             with open(filename, 'r',errors='ignore') as data_file:
#                 data=data_file.read().replace('\n', '')
#                 data = re.sub(r"[,:.;@#\?\-\%\/!&$]+\ *", " ", data)
#                 data = ''.join(filter(whitelist.__contains__, data)).lower()
#                 data=' '.join(data.split())
#             myfile.write(title+" "+data+ '\n')
#         else:
#             myfile.write(title+ '\n')

# with open('test_full.txt', 'w',errors='ignore') as myfile:
#     for i in range(len(test_data['title'])):
#         filename='raw_test/'+str(i+1)+'.txt'
#         if os.path.exists(filename):
#             with open(filename, 'r',errors='ignore') as data_file:
#                 data=data_file.read().replace('\n', '')
#             myfile.write(test_data['title'][i]+" "+data+ '\n')
#         else:
#             myfile.write(test_data['title'][i]+ '\n')

# #v2 remove numbers and punctuations
# whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ')
# with open('test_full_clean.txt', 'w',errors='ignore') as myfile:
#     for i in range(len(test_data['title'])):
#         filename='raw_test/'+str(i+1)+'.txt'
#         title=test_data['title'][i]
#         title = re.sub(r"[,:.;@#\?\-\%\/!&$]+\ *", " ", title)
#         title = ''.join(filter(whitelist.__contains__, title)).lower()
#         title = ' '.join(title.split())
#         if os.path.exists(filename):
#             with open(filename, 'r',errors='ignore') as data_file:
#                 data=data_file.read().replace('\n', '')
#                 data = re.sub(r"[,:.;@#\?\-\%\/!&$]+\ *", " ", data)
#                 data = ''.join(filter(whitelist.__contains__, data)).lower()
#                 data=' '.join(data.split())
#             myfile.write(title +" "+data+ '\n')
#         else:
#             myfile.write(title + '\n')

# Do classification

In [6]:
#trainf='train_full_clean.txt'
trainf='train_full.txt'
#X = pd.read_csv(trainf,header=None,sep=",",names=['data'])
with open(trainf) as f:
    X = f.read().splitlines()

df = pd.DataFrame({"data": X})
df.head()

,data
0,Forex - Pound drops to one-month lows against ...
1,Hertz to Exit Equipment Rental Business in $2....
2,Gold ETF inflows return Stock-market capital f...
3,"Hackers call Mt. Gox CEO a liar, say he still ..."
4,Gold Climbs To Near 6-Month High On Concerns A...


In [7]:
df = df.assign(article_id=train_data.article_id, category=train_data.category)

In [8]:
df.head(1)

,data,article_id,category
0,Forex - Pound drops to one-month lows against ...,1,4


In [9]:
warn_filename_train = 'raw_train/warn.txt'
error_filename_train = 'raw_train/error2.txt'
warn_filename_test = 'raw_test/warn.txt'
error_filename_test = 'raw_test/error2.txt'

In [10]:
def read_warn_or_error_list(filename):
    with open(filename, errors='ignore') as f:
        lines = f.read().splitlines()
    _list = []
    for line in lines:
        _list.append(line.split(',')[0])
    return _list

In [11]:
train_warn_list = read_warn_or_error_list(warn_filename_train)
train_error_list = read_warn_or_error_list(error_filename_train)
test_warn_list = read_warn_or_error_list(warn_filename_test)
test_error_list = read_warn_or_error_list(error_filename_test)

In [12]:
df_noerror = df[~df.article_id.isin(train_warn_list)]
# df_noerror = df[~df.article_id.isin(train_error_list)]

In [13]:
df_noerror.shape

(5211, 3)

In [25]:
X_train, X_test, Y_train, Y_test = train_test_split(df_noerror["data"], df_noerror["category"], test_size = 0.2)
X_train = np.array(X_train)
X_test = np.array(X_test)  # indeed X_valid is a more accurate name
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)  # indeed Y_valid is a more accurate name

In [26]:
def train_model(classifier, feature_vector_train, label_train, feature_vector_valid, label_valid):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label_train)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    return metrics.accuracy_score(predictions, label_valid)

In [69]:
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1,3), max_features=3000)
tfidf_vect_ngram.fit(df_noerror["data"])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(X_train)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(X_test)

/Users/ruizhi/anaconda/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [70]:
# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(LogisticRegression(penalty='l2', C=1.25, solver='lbfgs'), xtrain_tfidf_ngram, Y_train, xvalid_tfidf_ngram, Y_test)
print("LR, N-Gram Vectors: ", accuracy)

LR, N-Gram Vectors:  0.6826462128475551


In [71]:
# Naive Bayes Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(MultinomialNB(alpha=0.8), xtrain_tfidf_ngram, Y_train, xvalid_tfidf_ngram, Y_test)
print("NB, N-Gram Vectors: ", accuracy)

NB, N-Gram Vectors:  0.6586768935762224


In [30]:
# Extereme Gradient Boosting on Word Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(max_depth=6, learning_rate=0.1, n_estimators=160), xtrain_tfidf_ngram.tocsc(), Y_train, xvalid_tfidf_ngram.tocsc(), Y_test)
print("Xgb, WordLevel TF-IDF: ", accuracy)

Xgb, WordLevel TF-IDF:  0.6711409395973155


/Users/ruizhi/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [49]:
# Random Forest on Ngram Level TF IDF Vectors
accuracy = train_model(RandomForestClassifier(n_estimators=200, criterion='entropy'), xtrain_tfidf_ngram, Y_train, xvalid_tfidf_ngram, Y_test)
print("RF, N-Gram Vectors: ", accuracy)

RF, N-Gram Vectors:  0.653910149750416


In [52]:
# It seems that by filtering out those in the train_warn_list, the accuracy obtained is higher.

# Create Submission Using Different Mothods

In [152]:
#trainf='train_full_clean.txt'
trainf='test_full.txt'
#X = pd.read_csv(trainf,header=None,sep=",",names=['data'])
with open(trainf) as f:
    X = f.read().splitlines()

df_test = pd.DataFrame({"data": X})
df_test = df_test.assign(article_id=test_data.article_id)
df_test.head(1)

,data,article_id
0,White House plays down speedy role for US natu...,1


In [153]:
# filter out those with warnings
df_test = df_test[~df_test.article_id.isin(test_warn_list)]

In [154]:
df_test.shape

(3344, 2)

In [155]:
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1,3), max_features=3000)
tfidf_vect_ngram.fit(df_noerror["data"])

train_tfidf_ngram =  tfidf_vect_ngram.transform(df_noerror["data"])
test_tfidf_ngram =  tfidf_vect_ngram.transform(df_test["data"])

/Users/ruizhi/anaconda/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [156]:
classifier = LogisticRegression(penalty='l2', C=1.25, solver='lbfgs')
accuracy = train_model(classifier, train_tfidf_ngram, df_noerror["category"], train_tfidf_ngram, df_noerror["category"])
print("LR on training set ", accuracy)

LR on training set  0.7286509307234695


In [ ]:
# Naive Bayes Classifier on Ngram Level TF IDF Vectors
classifier =  MultinomialNB()
accuracy = train_model(classifier, train_tfidf_ngram, df_noerror["category"], train_tfidf_ngram, df_noerror["category"])
print("NB on training set ", accuracy)

In [157]:
classifier = xgboost.XGBClassifier(max_depth=5, learning_rate=0.1, n_estimators=140)
accuracy = train_model(classifier, train_tfidf_ngram, df_noerror["category"], train_tfidf_ngram, df_noerror["category"])
print("XGB on training set ", accuracy)

XGB on training set  0.8842832469775475


/Users/ruizhi/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [163]:
classifier = RandomForestClassifier(n_estimators=180, criterion='entropy')
accuracy = train_model(classifier, train_tfidf_ngram, df_noerror["category"], train_tfidf_ngram, df_noerror["category"])
print("RF on training set ", accuracy)

RF on training set  0.998272884283247


In [164]:
# test_data["category"] = classifier.predict(test_tfidf_ngram.tocsc())
df_test["category"] = classifier.predict(test_tfidf_ngram)

out = pd.DataFrame(df_test,columns=['article_id','category'])
out.to_csv('prediction_fulltext.csv',index=False)

# Update prediction by title

In [159]:
df_pred = pd.read_csv('prediction.csv')
df_pred_fulltext = pd.read_csv('prediction_fulltext.csv')

In [160]:
updated_id = []
updated_cat = []
num_diff = 0
for row in df_pred.itertuples():
    updated_id.append(row.article_id)
    cat_1 = row.category
    _ls = df_pred_fulltext[df_pred_fulltext.article_id==row.article_id].category.values
   
    if _ls.size > 0:  # if calculated in fulltext prediction
        cat_2 = _ls[0]
        # print(cat_1, cat_2)
        updated_cat.append(cat_2)
        if cat_1 != cat_2:
            num_diff += 1
    else:
        updated_cat.append(cat_1)

In [161]:
df_updated = pd.DataFrame({
    'article_id': updated_id,
    'category': updated_cat
})

In [162]:
df_updated.to_csv('prediction_updated.csv',index=False)